### Installing Dependencies

In [ ]:
!pip install cvzone

In [ ]:
!pip install mediapipe --user

### Importing the libraries

In [ ]:
import cv2 as cv
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

In [ ]:
cap=cv.VideoCapture(0)
detector=FaceMeshDetector(maxFaces=1)
plotY=LivePlot(640,360,[20,50],invert=True)
idList=[22,23,24,26,110,157,158,159,160,161,130,243]
dangerDetector=[]
driverSleeping=False
while True:
    success,img=cap.read()
    img=cv.flip(img,1)
    img, faces = detector.findFaceMesh(img,draw=False)
    if faces:
        face=faces[0]
        for id in idList:
            cv.circle(img,face[id],4,(255,0,255),cv.FILLED)
        leftUp=face[159]
        leftDown=face[23]
        leftLeft=face[130]
        leftRight=face[243]
        lengthVertical,_=detector.findDistance(leftUp,leftDown)
        lengthHorizontal,_=detector.findDistance(leftLeft,leftRight)
        cv.line(img,leftLeft,leftRight,(0,200,0),2)
        cv.line(img,leftUp,leftDown,(0,200,0),2)
        ratio=int(100*(lengthVertical/lengthHorizontal))
        if ratio<=29: #25
            success, img = cap.read()
            img = cv.flip(img, 1)
            img, faces = detector.findFaceMesh(img, draw=False)
            if faces:
                face = faces[0]
                for id in idList:
                    cv.circle(img, face[id], 4, (255, 0, 255), cv.FILLED)
                leftUp = face[159]
                leftDown = face[23]
                leftLeft = face[130]
                leftRight = face[243]
                lengthVertical, _ = detector.findDistance(leftUp, leftDown)
                lengthHorizontal, _ = detector.findDistance(leftLeft, leftRight)
                cv.line(img, leftLeft, leftRight, (0, 200, 0), 2)
                cv.line(img, leftUp, leftDown, (0, 200, 0), 2)
                ratio = int(100 * (lengthVertical / lengthHorizontal))
                if ratio>=31:
                    dangerDetector.clear()
                else:
                    dangerDetector.append(1)
        if len(dangerDetector)>=15:
            print("Driver is Sleeping!")
            driverSleeping=True          
        else:
            if driverSleeping:
                print("Driver is Awake!")
                driverSleeping=False
        imgPlot=plotY.update(ratio)
        imgPlot=cv.resize(imgPlot,(640,480))
    imgStack=cvzone.stackImages([img,imgPlot],2,1)
    cv.imshow("Image",imgStack)
    key=cv.waitKey(1)
    if key == ord('x'):
        break
cap.release()
cv.destroyAllWindows()